# ☸️ Kubernetes Fundamentals pour Data Engineers

Bienvenue dans ce module où tu vas apprendre à **orchestrer des containers à grande échelle** avec Kubernetes. Tu découvriras comment déployer, gérer et monitorer des applications data dans un cluster K8s — des compétences essentielles pour industrialiser tes pipelines !

---

## 📋 Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Avoir suivi le module `14_docker_for_data_engineers` |
| ✅ Requis | Maîtriser les bases de Docker (images, containers, volumes) |
| ✅ Requis | Connaissances de base en YAML |
| 💡 Recommandé | Docker Desktop ou Minikube installé |

## 🎯 Objectifs du module

À la fin de ce module, tu seras capable de :

- Comprendre l'architecture et les concepts clés de Kubernetes
- Installer et configurer un cluster Kubernetes local
- Déployer et gérer des applications avec `kubectl`
- Utiliser les Jobs et CronJobs pour des tâches Data Engineering
- Configurer le stockage persistant (PVC)
- Débugger des pods et diagnostiquer les erreurs courantes
- Déployer et orchestrer un pipeline ETL complet dans Kubernetes

---

## 🧠 C'est quoi Kubernetes ?

> ☸️ **Kubernetes (K8s)** est une plateforme open-source d'orchestration de containers qui automatise le déploiement, la mise à l'échelle et la gestion d'applications containerisées.

En pratique, Kubernetes te permet de :
- Déployer des containers sur **plusieurs machines** (un cluster)
- **Scaler** automatiquement selon la charge
- **Redémarrer** les containers qui plantent
- Gérer la **configuration** et les **secrets**
- Exposer tes applications au réseau

### 🔄 Kubernetes ≠ Docker (mais complémentaires)

| Aspect | Docker | Kubernetes |
|--------|--------|------------|
| **Rôle** | Créer et exécuter des containers | Orchestrer des containers |
| **Échelle** | 1 machine | Cluster de machines |
| **Focus** | Build & Run | Deploy, Scale, Manage |
| **Analogie** | Le musicien | Le chef d'orchestre |

> 💡 **Docker** crée les containers, **Kubernetes** les orchestre à grande échelle.

### 🎯 Analogies pour bien comprendre

| Analogie | Explication |
|----------|-------------|
| 🎼 **Chef d'orchestre** | K8s coordonne tous les containers (musiciens) pour qu'ils jouent en harmonie |
| 🏢 **Agence immobilière** | K8s place tes containers (locataires) dans les nodes (appartements) disponibles |
| ✈️ **Pilote automatique** | Tu définis la destination (état souhaité), K8s s'occupe d'y arriver et d'y rester |
| 🏭 **Usine automatisée** | Tu donnes les plans, K8s fabrique, surveille et remplace les pièces défectueuses |

> ℹ️ **Le savais-tu ?**
>
> Le nom **Kubernetes** vient du grec **κυβερνήτης** (kubernḗtēs) qui signifie "pilote" ou "gouverneur" — celui qui tient la barre d'un navire.
>
> K8s est né chez **Google**, inspiré de leur système interne **Borg** qui gère des milliards de containers depuis 2003. Google a ouvert le projet en 2014 et l'a donné à la **Cloud Native Computing Foundation (CNCF)**.
>
> Le "8" dans K8s représente les 8 lettres entre le K et le S de Kubernetes !
>
> 📖 [Histoire de Kubernetes](https://kubernetes.io/docs/concepts/overview/)

---

## 🚀 1. Pourquoi Kubernetes pour un Data Engineer ?

En Data Engineering moderne, tu dois souvent :

- Exécuter des **jobs ETL** de manière fiable et planifiée
- Déployer des **bases de données** et des **brokers** (Kafka)
- Faire tourner des jobs **Spark** distribués
- Orchestrer avec **Airflow** ou **Prefect**
- Scaler selon le volume de données

### ❌ Sans Kubernetes

| Problème | Conséquence |
|----------|-------------|
| Déploiement manuel sur chaque serveur | Lent et source d'erreurs |
| Pas de redémarrage automatique | Jobs perdus en cas de crash |
| Scaling manuel | Sous/sur-utilisation des ressources |
| Configuration dispersée | Difficile à maintenir |

### ✅ Avec Kubernetes

| Avantage | Exemple concret |
|----------|----------------|
| Déploiement déclaratif | `kubectl apply -f etl-job.yaml` |
| Auto-healing | Pod crashé = recréé automatiquement |
| CronJobs natifs | ETL planifié sans cron externe |
| Secrets management | Credentials gérés proprement |
| Portabilité | Même config en dev, staging, prod |

### 🖼️ Vue d'ensemble : Cluster K8s

```text
┌─────────────────────────────────────────────────────┐
│                   CLUSTER K8s                       │
│                                                     │
│   ┌─────────────┐   ┌─────────────┐                │
│   │   NODE 1    │   │   NODE 2    │                │
│   │  ┌───────┐  │   │  ┌───────┐  │                │
│   │  │ Pod A │  │   │  │ Pod C │  │                │
│   │  └───────┘  │   │  └───────┘  │                │
│   │  ┌───────┐  │   │  ┌───────┐  │                │
│   │  │ Pod B │  │   │  │ Pod D │  │                │
│   │  └───────┘  │   │  └───────┘  │                │
│   └─────────────┘   └─────────────┘                │
│                                                     │
└─────────────────────────────────────────────────────┘
```

---

## 🏗️ 2. Architecture Kubernetes (simplifiée)

Un cluster Kubernetes est composé de deux types de machines :

| Composant | Rôle | Analogie |
|-----------|------|----------|
| **Control Plane** (Master) | Cerveau du cluster, prend les décisions | Le management |
| **Worker Nodes** | Exécutent les containers | Les ouvriers |

### 🧠 Control Plane (les composants essentiels)

| Composant | Rôle |
|-----------|------|
| **API Server** | Point d'entrée unique (kubectl → API) |
| **Scheduler** | Décide sur quel node placer les pods |

> 💡 Pour ce module "Fundamentals", on se concentre sur ces 2 composants. Les autres (etcd, Controller Manager) seront vus en niveau avancé.

### 💪 Worker Nodes

| Composant | Rôle |
|-----------|------|
| **Kubelet** | Agent qui gère les pods sur le node |
| **Container Runtime** | Docker/containerd qui exécute les containers |

### 🖼️ Schéma Architecture

```text
                         ┌─────────────────────────────────────────┐
                         │           CONTROL PLANE                 │
                         │  ┌────────────────┐  ┌───────────────┐ │
       kubectl ─────────▶│  │   API Server   │  │   Scheduler   │ │
                         │  └───────┬────────┘  └───────────────┘ │
                         └──────────┼─────────────────────────────┘
                                    │
              ┌─────────────────────┼─────────────────────┐
              │                     │                     │
              ▼                     ▼                     ▼
┌─────────────────────┐ ┌─────────────────────┐ ┌─────────────────────┐
│     WORKER NODE 1   │ │     WORKER NODE 2   │ │     WORKER NODE 3   │
│  ┌───────────────┐  │ │  ┌───────────────┐  │ │  ┌───────────────┐  │
│  │    Kubelet    │  │ │  │    Kubelet    │  │ │  │    Kubelet    │  │
│  └───────────────┘  │ │  └───────────────┘  │ │  └───────────────┘  │
│  ┌─────┐  ┌─────┐   │ │  ┌─────┐  ┌─────┐   │ │  ┌─────┐           │
│  │ Pod │  │ Pod │   │ │  │ Pod │  │ Pod │   │ │  │ Pod │           │
│  └─────┘  └─────┘   │ │  └─────┘  └─────┘   │ │  └─────┘           │
└─────────────────────┘ └─────────────────────┘ └─────────────────────┘
```

---

## 🔑 3. Concepts clés Kubernetes

Voici les concepts fondamentaux à maîtriser :

| Concept | Description | Analogie |
|---------|-------------|----------|
| **Pod** | Plus petite unité déployable (1+ containers) | Un appartement |
| **Deployment** | Gère les replicas de pods, rolling updates | Un syndic d'immeuble |
| **Service** | Expose les pods sur le réseau | L'adresse postale |
| **Namespace** | Isolation logique des ressources | Un quartier de la ville |
| **ConfigMap** | Configuration externe (non sensible) | Fichier `.env` public |
| **Secret** | Données sensibles (base64) | Un coffre-fort |
| **PersistentVolumeClaim** | Demande de stockage persistant | Location d'un disque dur |
| **Job** | Tâche one-shot (s'exécute puis se termine) | Une mission ponctuelle |
| **CronJob** | Job planifié (comme cron Linux) | Un réveil programmé |

### 📦 Pod

Le **Pod** est l'unité de base dans Kubernetes :
- Contient **1 ou plusieurs containers** qui partagent le réseau et le stockage
- A sa propre **adresse IP** dans le cluster
- Est **éphémère** : peut être détruit et recréé à tout moment

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: mon-pod
spec:
  containers:
  - name: mon-container
    image: python:3.11-slim
```

### 🚀 Deployment

Le **Deployment** gère un ensemble de pods identiques :
- Garantit le **nombre de replicas** souhaité
- Gère les **rolling updates** (mise à jour sans downtime)
- Recrée les pods qui crashent

### 🌐 Service

Le **Service** expose les pods sur le réseau :

| Type | Description | Usage |
|------|-------------|-------|
| **ClusterIP** | IP interne au cluster | Communication entre pods |
| **NodePort** | Port ouvert sur chaque node | Tests, développement |
| **LoadBalancer** | Load balancer cloud | Production (AWS, GCP, Azure) |

### 🔐 Secret vs ConfigMap

| Aspect | ConfigMap | Secret |
|--------|-----------|--------|
| Usage | Config non sensible | Passwords, tokens, clés |
| Encodage | Texte clair | Base64 |
| Exemple | URLs, feature flags | DB_PASSWORD, API_KEY |

⚠️ **Attention : Base64 n'est PAS du chiffrement !**

> Base64 est juste un **encodage**, pas une protection. N'importe qui avec accès au cluster peut décoder les secrets.
> En production, utilise **Sealed Secrets**, **HashiCorp Vault**, ou les secrets managers cloud (AWS Secrets Manager, GCP Secret Manager).

### 🖼️ Schéma : Relations entre concepts

```text
                    ┌──────────────┐
       User ───────▶│   Service    │
                    └──────┬───────┘
                           │
                           ▼
                    ┌──────────────┐
                    │  Deployment  │
                    └──────┬───────┘
                           │
              ┌────────────┼────────────┐
              ▼            ▼            ▼
         ┌───────┐    ┌───────┐    ┌───────┐
         │ Pod 1 │    │ Pod 2 │    │ Pod 3 │
         └───┬───┘    └───┬───┘    └───┬───┘
             │            │            │
             ▼            ▼            ▼
        Container    Container    Container
```

---

## 💻 4. Installation de Kubernetes

Pour ce module, tu as besoin d'un cluster Kubernetes local.

| Option | Difficulté | Recommandé pour |
|--------|------------|----------------|
| **Docker Desktop** | ⭐ Facile | Mac/Windows, débutants |
| **Minikube** | ⭐⭐ Moyen | Tous OS, plus de contrôle |
| **k3d/k3s** | ⭐⭐ Moyen | Léger, CI/CD |

### 🐳 Option 1 : Docker Desktop (recommandé)

1. Ouvrir **Docker Desktop**
2. Aller dans **Settings** → **Kubernetes**
3. Cocher **Enable Kubernetes**
4. Cliquer **Apply & Restart**
5. Attendre que le status passe au vert ✅

### 📦 Option 2 : Minikube

```bash
# Installation (macOS avec Homebrew)
brew install minikube

# Installation (Linux)
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

# Démarrer le cluster
minikube start

# Vérifier
minikube status
```

### 🔧 Installer kubectl

`kubectl` est l'outil CLI pour interagir avec Kubernetes.

```bash
# macOS
brew install kubectl

# Linux
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
sudo install kubectl /usr/local/bin/kubectl

# Windows (avec Docker Desktop, kubectl est inclus)
```

### ✅ Vérification

```bash
# Version de kubectl
kubectl version --client

# Liste des nodes du cluster
kubectl get nodes

# Vérifier le contexte actif
kubectl config get-contexts
```

In [ ]:
%%bash
# Vérifier l'installation de kubectl
echo "=== Version kubectl ==="
kubectl version --client

echo ""
echo "=== Nodes du cluster ==="
kubectl get nodes

echo ""
echo "=== Contexte actif ==="
kubectl config current-context

---

## 🛠️ 5. Commandes kubectl essentielles

Voici les commandes que tu utiliseras au quotidien :

### 📋 CRUD sur les ressources

| Action | Commande |
|--------|----------|
| Créer/Mettre à jour | `kubectl apply -f manifest.yaml` |
| Lire | `kubectl get pods` |
| Détails | `kubectl describe pod <name>` |
| Supprimer | `kubectl delete -f manifest.yaml` |

### 🔍 Lister les ressources

```bash
kubectl get pods                    # Lister les pods
kubectl get pods -o wide            # Avec plus de détails (node, IP)
kubectl get deployments             # Lister les deployments
kubectl get services                # Lister les services
kubectl get all                     # Tout lister
kubectl get all -n <namespace>      # Dans un namespace spécifique
```

### 🐛 Debug

```bash
kubectl logs <pod>                  # Voir les logs
kubectl logs -f <pod>               # Suivre les logs en temps réel
kubectl logs -p <pod>               # Logs du container précédent (après crash)
kubectl describe pod <pod>          # Détails + Events
kubectl exec -it <pod> -- bash      # Shell dans le pod
kubectl get events                  # Tous les events du namespace
```

### 🌐 Réseau

```bash
kubectl port-forward <pod> 8080:80           # Forward un port
kubectl port-forward svc/<service> 8080:80   # Forward depuis un service
```

### 📦 Namespaces

```bash
kubectl get namespaces              # Lister les namespaces
kubectl create namespace dev        # Créer un namespace
kubectl config set-context --current --namespace=dev  # Changer de namespace par défaut
```

### 🗂️ Cheat Sheet visuel

```text
┌─────────────────────────────────────────────────────────────────┐
│                    KUBECTL CHEAT SHEET                         │
├─────────────────────────────────────────────────────────────────┤
│  CREATE/UPDATE │  kubectl apply -f manifest.yaml               │
│  READ          │  kubectl get pods/deploy/svc/all              │
│  DESCRIBE      │  kubectl describe <resource> <name>           │
│  DELETE        │  kubectl delete -f manifest.yaml              │
├─────────────────────────────────────────────────────────────────┤
│  LOGS          │  kubectl logs -f <pod>                        │
│  SHELL         │  kubectl exec -it <pod> -- bash               │
│  PORT-FORWARD  │  kubectl port-forward <pod> 8080:80           │
│  EVENTS        │  kubectl get events --sort-by='.lastTimestamp'│
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
%%bash
# Exemples de commandes kubectl

echo "=== Namespaces ==="
kubectl get namespaces

echo ""
echo "=== Pods dans tous les namespaces ==="
kubectl get pods --all-namespaces

echo ""
echo "=== Services ==="
kubectl get services

---

## 🎯 6. Premier déploiement : Hello World

Déployons une application simple pour comprendre le cycle de déploiement.

### 📝 Manifest YAML : Deployment + Service

```yaml
# hello-world.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world
  labels:
    app: hello-world
spec:
  replicas: 2
  selector:
    matchLabels:
      app: hello-world
  template:
    metadata:
      labels:
        app: hello-world
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        ports:
        - containerPort: 80
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-svc
spec:
  selector:
    app: hello-world
  ports:
  - port: 80
    targetPort: 80
  type: NodePort
```

### 🚀 Déployer

```bash
# Appliquer le manifest
kubectl apply -f hello-world.yaml

# Vérifier le déploiement
kubectl get deployments
kubectl get pods
kubectl get services
```

### 🌐 Accéder à l'application

**Méthode 1 : Port-forward (recommandé)**

```bash
kubectl port-forward svc/hello-world-svc 8080:80
# Ouvrir http://localhost:8080
```

**Méthode 2 : NodePort**

```bash
# Trouver le NodePort
kubectl get svc hello-world-svc
# Accéder via http://localhost:<NodePort>
```

> 💡 **Astuce entreprise** : Le NodePort n'est pas toujours autorisé en entreprise (firewall). `port-forward` fonctionne partout !

### 🔄 Cycle de déploiement

```text
kubectl apply     Deployment créé     ReplicaSet créé     Pods créés
     │                  │                   │                  │
     ▼                  ▼                   ▼                  ▼
┌─────────┐       ┌───────────┐       ┌───────────┐      ┌─────────┐
│  YAML   │ ───▶  │Deployment │ ───▶  │ReplicaSet │ ───▶ │  Pods   │
└─────────┘       └───────────┘       └───────────┘      └─────────┘
                                                               │
                    Service expose les pods ◀──────────────────┘
```

### 🧹 Nettoyage

```bash
kubectl delete -f hello-world.yaml
```

---

## 🗄️ 7. Kubernetes pour Data Engineering

Voici les ressources Kubernetes particulièrement utiles pour le Data Engineering.

### 7.1 Job : tâche one-shot

Un **Job** exécute un ou plusieurs pods jusqu'à **complétion** :
- ETL ponctuel
- Migration de données
- Backfill

```yaml
# etl-job.yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: etl-job
spec:
  template:
    spec:
      containers:
      - name: etl
        image: python:3.11-slim
        command: ["python", "-c", "print('ETL terminé !')"]
      restartPolicy: Never
  backoffLimit: 3  # Nombre de retries en cas d'échec
```

```bash
# Lancer le job
kubectl apply -f etl-job.yaml

# Voir le status
kubectl get jobs

# Voir les logs
kubectl logs job/etl-job
```

---

### 7.2 CronJob : tâche planifiée

Un **CronJob** lance des Jobs selon un schedule (comme cron Linux) :
- ETL quotidien
- Nettoyage de données
- Rapports automatisés

```yaml
# etl-cronjob.yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: etl-daily
spec:
  schedule: "0 2 * * *"  # Tous les jours à 2h du matin
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: etl
            image: my-etl-image:1.0
            env:
            - name: DB_HOST
              value: "postgres"
          restartPolicy: OnFailure
  successfulJobsHistoryLimit: 3
  failedJobsHistoryLimit: 1
```

**Schedule cron :**

| Expression | Signification |
|------------|---------------|
| `0 2 * * *` | Tous les jours à 2h00 |
| `*/15 * * * *` | Toutes les 15 minutes |
| `0 0 * * 0` | Tous les dimanches à minuit |
| `0 8 1 * *` | Le 1er de chaque mois à 8h |

```bash
# Lister les cronjobs
kubectl get cronjobs

# Déclencher manuellement
kubectl create job --from=cronjob/etl-daily test-etl
```

---

### 7.3 Déployer PostgreSQL dans K8s

```yaml
# postgres-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: postgres
spec:
  replicas: 1
  selector:
    matchLabels:
      app: postgres
  template:
    metadata:
      labels:
        app: postgres
    spec:
      containers:
      - name: postgres
        image: postgres:16
        ports:
        - containerPort: 5432
        env:
        - name: POSTGRES_USER
          value: "de_user"
        - name: POSTGRES_PASSWORD
          valueFrom:
            secretKeyRef:
              name: postgres-secret
              key: password
        - name: POSTGRES_DB
          value: "de_db"
        volumeMounts:
        - name: postgres-storage
          mountPath: /var/lib/postgresql/data
      volumes:
      - name: postgres-storage
        persistentVolumeClaim:
          claimName: postgres-pvc
```

> ⚠️ **Note production** : Les bases de données dans K8s nécessitent **toujours** un stockage persistant (PVC) (A voir plus en bas).
> En production, on préfère souvent des **services managés** (Cloud SQL, RDS, Azure Database) pour la haute disponibilité et les backups automatiques.

### 🖼️ Schéma Pipeline Data Engineering sur K8s

```text
┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│   CronJob    │────▶│     Job      │────▶│     Pod      │
│  (0 2 * * *) │     │              │     │  (etl.py)    │
└──────────────┘     └──────────────┘     └──────┬───────┘
                                                  │
                          ┌───────────────────────┤
                          │                       │
                          ▼                       ▼
                   ┌──────────────┐       ┌──────────────┐
                   │     PVC      │       │  PostgreSQL  │
                   │ (input data) │       │   (output)   │
                   └──────────────┘       └──────────────┘
```

---

## 💾 8. Stockage : Volumes & PVC

Les données ne doivent **jamais** vivre uniquement dans un Pod (éphémère).

### 🔑 Concepts clés

| Concept | Description |
|---------|-------------|
| **PersistentVolume (PV)** | Ressource de stockage provisionnée par l'admin |
| **PersistentVolumeClaim (PVC)** | Demande de stockage par l'utilisateur |
| **StorageClass** | Type de stockage (SSD, HDD, cloud...) |

### 🖼️ Schéma : Comment ça marche

```text
┌─────────┐     ┌─────────┐     ┌─────────────┐     ┌──────────────────┐
│   Pod   │────▶│   PVC   │────▶│     PV      │────▶│  Stockage réel   │
│         │     │ (claim) │     │ (provision) │     │ (disk, NFS, EBS) │
└─────────┘     └─────────┘     └─────────────┘     └──────────────────┘
```

### 📝 Exemple : PVC pour PostgreSQL

```yaml
# postgres-pvc.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: postgres-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 5Gi
```

**Access Modes :**

| Mode | Description |
|------|-------------|
| `ReadWriteOnce` | Lecture/écriture par un seul node |
| `ReadOnlyMany` | Lecture seule par plusieurs nodes |
| `ReadWriteMany` | Lecture/écriture par plusieurs nodes |

### 🔗 Utilisation dans un Deployment

```yaml
spec:
  containers:
  - name: postgres
    volumeMounts:
    - name: postgres-storage
      mountPath: /var/lib/postgresql/data
  volumes:
  - name: postgres-storage
    persistentVolumeClaim:
      claimName: postgres-pvc
```

```bash
# Vérifier les PVC
kubectl get pvc

# Vérifier les PV
kubectl get pv
```

---

## 🔐 9. Configuration : ConfigMaps & Secrets

### 📄 ConfigMap : configuration non sensible

```yaml
# config.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: etl-config
data:
  DB_HOST: "postgres"
  DB_PORT: "5432"
  LOG_LEVEL: "INFO"
```

### 🔒 Secret : données sensibles

```yaml
# secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: postgres-secret
type: Opaque
data:
  password: ZGVfcGFzc3dvcmQ=  # base64 de "de_password"
```

**Créer un secret en ligne de commande :**

```bash
# Créer un secret
kubectl create secret generic postgres-secret \
  --from-literal=password=de_password

# Encoder en base64
echo -n "de_password" | base64

# Décoder
echo "ZGVfcGFzc3dvcmQ=" | base64 -d
```

### 🔗 Utilisation dans un Pod

```yaml
spec:
  containers:
  - name: etl
    env:
    # Depuis ConfigMap
    - name: DB_HOST
      valueFrom:
        configMapKeyRef:
          name: etl-config
          key: DB_HOST
    # Depuis Secret
    - name: DB_PASSWORD
      valueFrom:
        secretKeyRef:
          name: postgres-secret
          key: password
```

---

## 🔧 10. Debug & Monitoring

### 📜 Commandes de debug essentielles

```bash
# Logs
kubectl logs <pod>                  # Logs actuels
kubectl logs -f <pod>               # Suivre en temps réel
kubectl logs -p <pod>               # Logs du container précédent (après crash)
kubectl logs <pod> -c <container>   # Logs d'un container spécifique

# Détails et Events
kubectl describe pod <pod>          # Détails complets + Events
kubectl get events --sort-by='.lastTimestamp'  # Events triés par date

# Shell dans le pod
kubectl exec -it <pod> -- bash      # Ouvrir un shell
kubectl exec -it <pod> -- sh        # Pour images Alpine

# Ressources
kubectl top pods                    # CPU/RAM des pods (si metrics-server)
kubectl top nodes                   # CPU/RAM des nodes
```

### ❌ Erreurs classiques et solutions

| Erreur | Cause probable | Solution |
|--------|---------------|----------|
| `CrashLoopBackOff` | L'app plante au démarrage | `kubectl logs <pod>` pour voir l'erreur |
| `ImagePullBackOff` | Image introuvable ou accès refusé | Vérifier nom/tag, credentials registry |
| `Pending` | Pas de ressources disponibles | `kubectl describe pod` → voir Events |
| `OOMKilled` | Mémoire insuffisante | Augmenter `resources.limits.memory` |
| `CreateContainerConfigError` | ConfigMap/Secret manquant | Vérifier que les refs existent |
| `ErrImageNeverPull` | Image locale introuvable | `imagePullPolicy: IfNotPresent` |

### 🔍 Processus de debug

```text
1. kubectl get pods              → Voir le status
           │
           ▼
2. kubectl describe pod <pod>    → Voir les Events
           │
           ▼
3. kubectl logs <pod>            → Voir les logs de l'app
           │
           ▼
4. kubectl exec -it <pod> -- sh  → Investiguer dans le container
```

---

## ⚠️ 11. Erreurs fréquentes & Bonnes pratiques

### ❌ Erreurs fréquentes des débutants

| Erreur | Conséquence |
|--------|-------------|
| Pas de `resources.requests/limits` | Pods non schedulés ou OOMKilled |
| Secrets dans ConfigMap | Fuite de credentials |
| Base de données sans PVC | Perte de données au restart |
| Tout dans namespace `default` | Difficile à gérer, pas d'isolation |
| Images en `:latest` | Comportement non reproductible |
| Pas de labels | Impossible de filtrer/organiser |

### ✅ Bonnes pratiques

| Pratique | Pourquoi |
|----------|----------|
| **Toujours définir requests/limits** | Scheduling prévisible, protection contre OOM |
| **Un namespace par projet/env** | Isolation, quotas, RBAC |
| **Tags d'image versionnés** | `image:1.0.0` au lieu de `:latest` |
| **Labels systématiques** | Organisation et filtrage |
| **Healthchecks (probes)** | K8s sait si l'app est prête/vivante |
| **PVC pour toute donnée importante** | Persistance garantie |

### 🏷️ Exemple de labels recommandés

```yaml
metadata:
  name: etl-job
  labels:
    app: etl-pipeline
    component: etl
    env: dev
    team: data-engineering
    version: "1.0.0"
```

### 📊 Exemple de resources requests/limits

```yaml
spec:
  containers:
  - name: etl
    resources:
      requests:
        memory: "256Mi"
        cpu: "250m"
      limits:
        memory: "512Mi"
        cpu: "500m"
```

| Ressource | Unité | Exemple |
|-----------|-------|--------|
| **CPU** | millicores | `500m` = 0.5 CPU |
| **Memory** | Mi/Gi | `256Mi`, `1Gi` |

---

## 🧪 Quiz de fin de module

Réponds aux questions suivantes pour vérifier tes acquis.

---

### ❓ Q1. Quelle est la différence entre un Pod et un Deployment ?
a) Un Pod contient plusieurs Deployments  
b) Un Deployment gère des replicas de Pods et leur cycle de vie  
c) C'est la même chose  
d) Un Pod est pour la production, un Deployment pour le dev

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Un Pod est l'unité de base (éphémère), un Deployment gère plusieurs replicas de Pods, les rolling updates, et recrée les Pods qui crashent.

</details>

---

### ❓ Q2. Pourquoi utiliser un PersistentVolumeClaim (PVC) ?
a) Pour augmenter la vitesse du CPU  
b) Pour persister les données même si le Pod est supprimé  
c) Pour exposer un Pod sur internet  
d) Pour chiffrer les secrets

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Les Pods sont éphémères. Sans PVC, les données sont perdues quand le Pod est supprimé ou recréé.

</details>

---

### ❓ Q3. Quelle commande permet de voir les logs d'un Pod qui a crashé ?
a) `kubectl logs <pod>`  
b) `kubectl logs -p <pod>`  
c) `kubectl describe pod <pod>`  
d) `kubectl get logs <pod>`

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — L'option `-p` (previous) affiche les logs du container précédent, utile après un crash.

</details>

---

### ❓ Q4. L'encodage Base64 des Secrets Kubernetes est-il sécurisé ?
a) Oui, c'est du chiffrement fort  
b) Non, Base64 est juste un encodage, pas du chiffrement  
c) Oui, si on utilise une clé de 256 bits  
d) Ça dépend de la version de Kubernetes

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Base64 est un simple encodage réversible. N'importe qui avec accès au cluster peut décoder les secrets. En production, utilisez Sealed Secrets ou un secrets manager.

</details>

---

### ❓ Q5. Quelle ressource utiliser pour exécuter un ETL tous les jours à 2h du matin ?
a) Deployment  
b) Job  
c) CronJob  
d) DaemonSet

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : c** — Un CronJob permet de planifier des Jobs selon une expression cron (`0 2 * * *`).

</details>

---

### ❓ Q6. Que signifie l'erreur `CrashLoopBackOff` ?
a) L'image Docker n'existe pas  
b) Le Pod manque de mémoire  
c) L'application dans le container plante au démarrage de manière répétée  
d) Le réseau est inaccessible

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : c** — K8s tente de redémarrer le container mais il plante à chaque fois. Utilisez `kubectl logs <pod>` pour voir l'erreur.

</details>

---

### ❓ Q7. Quelle est la meilleure pratique pour les tags d'images en production ?
a) Toujours utiliser `:latest`  
b) Utiliser des tags versionnés comme `:1.0.0`  
c) Ne pas mettre de tag  
d) Utiliser la date comme tag

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Les tags versionnés garantissent la reproductibilité. `:latest` peut changer et causer des comportements inattendus.

</details>

---

## 🚀 Mini-projet : Pipeline ETL sur Kubernetes

### 🎯 Objectif
Déployer un pipeline ETL complet sur Kubernetes :
- **PostgreSQL** avec stockage persistant
- **CronJob** Python qui charge des données CSV dans PostgreSQL

### 🔧 Contexte
Tu dois créer une stack Data Engineering minimale mais réaliste, entièrement orchestrée par Kubernetes.

### 📁 Structure du projet

```text
k8s-etl-project/
├── manifests/
│   ├── namespace.yaml
│   ├── postgres-secret.yaml
│   ├── postgres-pvc.yaml
│   ├── postgres-deployment.yaml
│   ├── postgres-service.yaml
│   ├── etl-configmap.yaml
│   └── etl-cronjob.yaml
├── etl/
│   ├── Dockerfile
│   ├── requirements.txt
│   └── etl.py
├── data/
│   └── sales.csv
└── README.md
```

### 🧠 Étapes

1. **Créer le namespace** `data-pipeline`
2. **Déployer PostgreSQL** (Secret + PVC + Deployment + Service)
3. **Build & push** l'image Docker de l'ETL
4. **Déployer le CronJob** ETL
5. **Tester manuellement** : `kubectl create job --from=cronjob/etl-daily test-etl`
6. **Vérifier** les données dans PostgreSQL

### ✅ Solution du mini-projet

<details>
<summary>📥 Afficher la solution complète</summary>

**1. `manifests/namespace.yaml`**
```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: data-pipeline
  labels:
    team: data-engineering
```

**2. `manifests/postgres-secret.yaml`**
```yaml
apiVersion: v1
kind: Secret
metadata:
  name: postgres-secret
  namespace: data-pipeline
type: Opaque
data:
  password: ZGVfcGFzc3dvcmQ=  # de_password
```

**3. `manifests/postgres-pvc.yaml`**
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: postgres-pvc
  namespace: data-pipeline
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
```

**4. `manifests/postgres-deployment.yaml`**
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: postgres
  namespace: data-pipeline
  labels:
    app: postgres
spec:
  replicas: 1
  selector:
    matchLabels:
      app: postgres
  template:
    metadata:
      labels:
        app: postgres
    spec:
      containers:
      - name: postgres
        image: postgres:16
        ports:
        - containerPort: 5432
        env:
        - name: POSTGRES_USER
          value: "de_user"
        - name: POSTGRES_PASSWORD
          valueFrom:
            secretKeyRef:
              name: postgres-secret
              key: password
        - name: POSTGRES_DB
          value: "de_db"
        volumeMounts:
        - name: postgres-storage
          mountPath: /var/lib/postgresql/data
        resources:
          requests:
            memory: "256Mi"
            cpu: "250m"
          limits:
            memory: "512Mi"
            cpu: "500m"
      volumes:
      - name: postgres-storage
        persistentVolumeClaim:
          claimName: postgres-pvc
```

**5. `manifests/postgres-service.yaml`**
```yaml
apiVersion: v1
kind: Service
metadata:
  name: postgres
  namespace: data-pipeline
spec:
  selector:
    app: postgres
  ports:
  - port: 5432
    targetPort: 5432
  type: ClusterIP
```

**6. `manifests/etl-configmap.yaml`**
```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: etl-config
  namespace: data-pipeline
data:
  DB_HOST: "postgres"
  DB_PORT: "5432"
  DB_NAME: "de_db"
  DB_USER: "de_user"
```

**7. `manifests/etl-cronjob.yaml`**
```yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: etl-daily
  namespace: data-pipeline
  labels:
    app: etl-pipeline
spec:
  schedule: "0 2 * * *"
  jobTemplate:
    spec:
      template:
        metadata:
          labels:
            app: etl-job
        spec:
          containers:
          - name: etl
            image: my-etl-image:1.0  # À remplacer par ton image
            envFrom:
            - configMapRef:
                name: etl-config
            env:
            - name: DB_PASSWORD
              valueFrom:
                secretKeyRef:
                  name: postgres-secret
                  key: password
            resources:
              requests:
                memory: "128Mi"
                cpu: "100m"
              limits:
                memory: "256Mi"
                cpu: "200m"
          restartPolicy: OnFailure
  successfulJobsHistoryLimit: 3
  failedJobsHistoryLimit: 1
```

**8. `etl/etl.py`**
```python
import os
import pandas as pd
from sqlalchemy import create_engine

def main():
    # Config depuis variables d'environnement
    db_host = os.environ['DB_HOST']
    db_port = os.environ['DB_PORT']
    db_name = os.environ['DB_NAME']
    db_user = os.environ['DB_USER']
    db_pass = os.environ['DB_PASSWORD']
    
    print("🚀 Démarrage ETL...")
    
    # Simuler des données (en vrai: lire depuis PVC ou S3)
    df = pd.DataFrame({
        'date': ['2024-01-01', '2024-01-02'],
        'product': ['Laptop', 'Mouse'],
        'quantity': [5, 20],
        'price': [999.99, 29.99]
    })
    df['total'] = df['quantity'] * df['price']
    
    # Charger dans PostgreSQL
    engine = create_engine(
        f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}'
    )
    df.to_sql('sales', engine, if_exists='replace', index=False)
    
    print("✅ ETL terminé !")

if __name__ == "__main__":
    main()
```

**9. Commandes de déploiement**
```bash
# Créer le namespace
kubectl apply -f manifests/namespace.yaml

# Déployer PostgreSQL
kubectl apply -f manifests/postgres-secret.yaml
kubectl apply -f manifests/postgres-pvc.yaml
kubectl apply -f manifests/postgres-deployment.yaml
kubectl apply -f manifests/postgres-service.yaml

# Vérifier que PostgreSQL est prêt
kubectl get pods -n data-pipeline

# Déployer le CronJob
kubectl apply -f manifests/etl-configmap.yaml
kubectl apply -f manifests/etl-cronjob.yaml

# Tester manuellement
kubectl create job --from=cronjob/etl-daily test-etl -n data-pipeline

# Voir les logs
kubectl logs -f job/test-etl -n data-pipeline

# Vérifier dans PostgreSQL
kubectl exec -it $(kubectl get pod -l app=postgres -n data-pipeline -o name) \
  -n data-pipeline -- psql -U de_user -d de_db -c "SELECT * FROM sales;"
```

</details>

---

## 📚 Ressources pour aller plus loin

### 🌐 Documentation officielle
- [Kubernetes Docs](https://kubernetes.io/docs/) — Documentation officielle
- [kubectl Cheat Sheet](https://kubernetes.io/docs/reference/kubectl/cheatsheet/) — Toutes les commandes
- [Kubernetes API Reference](https://kubernetes.io/docs/reference/kubernetes-api/) — Référence API

### 🎮 Pratique
- [Katacoda K8s](https://www.katacoda.com/courses/kubernetes) — Tutoriels interactifs (gratuit)
- [Play with Kubernetes](https://labs.play-with-k8s.com/) — Cluster K8s gratuit en ligne
- [Killer.sh](https://killer.sh/) — Simulateur d'examen CKA/CKAD

### 📖 Livres & Cours
- *Kubernetes Up & Running* — Kelsey Hightower
- *The Kubernetes Book* — Nigel Poulton

### 🔧 Outils utiles
- [k9s](https://k9scli.io/) — Terminal UI pour Kubernetes
- [Lens](https://k8slens.dev/) — IDE Kubernetes
- [kubectx/kubens](https://github.com/ahmetb/kubectx) — Changer de contexte/namespace facilement

---

## ➡️ Prochaine étape

Maintenant que tu maîtrises les fondamentaux de Kubernetes, passons aux **workloads Data Engineering avancés** !

👉 **Module suivant : `16_k8s_for_data_workloads.ipynb`** — Kubernetes pour les workloads Data

Tu vas apprendre :
- Spark on Kubernetes
- Airflow on Kubernetes
- Helm charts pour packager tes déploiements
- Horizontal Pod Autoscaler

---

🎉 **Félicitations !** Tu as terminé le module Kubernetes Fundamentals pour Data Engineers.